In [5]:
import random
# import pandas as pd
# from IPython.display import display, HTML

import torch
from torch import utils
from torch.utils import data
from torch.utils.data import Dataset, DataLoader

import multiprocessing

import accelerate
from accelerate import Accelerator

# import huggingface_hub
from transformers import T5ForConditionalGeneration, T5Config, T5TokenizerFast, T5Tokenizer, AutoTokenizer
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer, default_data_collator, get_scheduler
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


import datasets
from datasets import load_dataset #, load_from_disk
# import evaluate
# from evaluate import load


import tqdm as notebook_tqdm
import os
from dotenv import load_dotenv

from tqdm.auto import tqdm
import math

In [6]:
load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# huggingface_token = os.getenv("HF_TOKEN")

# device_type = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device_type)
# print(torch.cuda.device_count())
cpu_cores = multiprocessing.cpu_count()
# print(cpu_cores)
# device = torch.device("cpu")


accelerator = Accelerator()
accelerator.print("test")
accelerator.print(f"n_gpu: {accelerator}")


test
n_gpu: <accelerate.accelerator.Accelerator object at 0x7f6594e70410>


In [7]:
model_checkpoint = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
# model = model.to(device)

In [8]:
raw_dataset = load_dataset("xsum")

In [9]:
train_size = 1000
test_size = int(0.1 * train_size)
downsampled_dataset = raw_dataset["train"].train_test_split(train_size=train_size, test_size=test_size)

In [10]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 128

In [11]:
# def preprocess_examples(examples):
#   # encode the code-docstring pairs
#   codes = examples['code']
#   docstrings = examples['docstring']
  
#   inputs = [prefix + code for code in codes]
#   model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

#   # encode the summaries
#   labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

#   # important: we need to replace the index of the padding tokens by -100
#   # such that they are not taken into account by the CrossEntropyLoss
#   labels_with_ignore_index = []
#   for labels_example in labels:
#     labels_example = [label if label != 0 else -100 for label in labels_example]
#     labels_with_ignore_index.append(labels_example)
  
#   model_inputs["labels"] = labels_with_ignore_index

#   return model_inputs

In [12]:
# dataset = raw_dataset.map(preprocess_examples, batched=True)

In [13]:
# @dataclass
# class T2TDataCollator(DataCollator):
#     def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
#         """
#         Take a list of samples from a Dataset and collate them into a batch.
#         Returns:
#             A dictionary of tensors
#         """
#         input_ids = torch.stack([example['input_ids'] for example in batch])
#         lm_labels = torch.stack([example['target_ids'] for example in batch])
#         lm_labels[lm_labels[:, :] == 0] = -100
#         attention_mask = torch.stack([example['attention_mask'] for example in batch])
#         decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

#         return {
#             'input_ids': input_ids, 
#             'attention_mask': attention_mask,
#             'lm_labels': lm_labels, 
#             'decoder_attention_mask': decoder_attention_mask
#         }


In [14]:
# prefix = "summarize: "
# max_input_length = 512
# max_target_length = 128



# Add prefix to start of each document in dataset. Do NOT use tokenizer() here!
def add_prefix_to_dataset(batch):
    batch["document"] = [prefix + doc for doc in batch["document"]]
    return batch

prefixed_downsampled_dataset = downsampled_dataset.map(add_prefix_to_dataset, batched=True, num_proc=cpu_cores)

train_dataset_untokenized = prefixed_downsampled_dataset["train"]
test_dataset_untokenized = prefixed_downsampled_dataset["test"]



Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [15]:
def tokenize_dataset(examples):
    model_inputs = tokenizer(examples['document'], max_length=max_input_length, truncation=True, padding="max_length", return_tensors="pt")

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True, padding="max_length", return_tensors="pt")
    
    labels.input_ids[labels.input_ids == tokenizer.pad_token_id] = -100
    
    # for key, value in labels.items():
    #     if torch.any(value == 0):
    #         labels[key][value == 0] = -100

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_datasets = prefixed_downsampled_dataset.map(tokenize_dataset, batched=True, num_proc=cpu_cores)



# remove columns 'documents' and 'summary' from tokenized_datasets
tokenized_datasets = tokenized_datasets.remove_columns(["document", "summary", "id"])
tokenized_datasets = tokenized_datasets.with_format("torch")

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'

Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'

In [16]:
print(type(tokenized_datasets['train']['input_ids']))
print(train_dataset[0])

# tester = train_dataset['input_ids']
# tester = torch.tensor(tester)  
# print(type(tester))

<class 'torch.Tensor'>
{'input_ids': tensor([21603,    10,  2575,  1685, 10256,    19, 17679,     8,   422, 22494,
          859,   151,    28,  2416,    12,   301,    40,   159,  3321,   651,
         1592,  1121,     5,   707,  3059, 23091,    44, 26911,    11,  3833,
           15,   533,  1476,   243,    95,  4914,    13,     8,  2662,   115,
           47,    96,  3216,    49,   145,    62,   133,   114,    34,    12,
           36,  1280,   216,   243,    48,  3679,   856, 22494,     7,   130,
           96,  2376,   120,  1280,     3, 31334,    15,  3975,    43,   641,
         1026,   286,    44,   301,    40,   159,  3321,   651,  1592,  1121,
           11,     8,  2329,  2061,    16, 24260,  5229,    12,    34,     5,
          707, 23091,     6,     3,     9,  7792,    16,   452,   533,  4404,
            6,   243,    10,    96,  1326,   214,    24,   300,    80,    16,
         2391,   502,    11,  1021,   151,    16, 26911,    11,     8,  3833,
           15,    13, 10941

In [17]:
print(model)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [18]:
# batch_size = 16
# model_name = model_checkpoint.split("/")[-1]
# args = Seq2SeqTrainingArguments(
#     f"{model_name}-finetuned-xsum",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=2,
#     predict_with_generate=True,
#     fp16=True,
#     push_to_hub=False,
# )

In [19]:
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

In [21]:
# trainer.train()

In [22]:

# print(train_dataset['input_ids'][0:5])

# for i in range(5):
#     print(len(train_dataset['input_ids'][i]))
#     print(len(train_dataset['labels'][i]))
    


In [23]:
batch_size = 64
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, max_length=max_input_length, return_tensors="pt")
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
training_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
eval_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

loss_function = torch.nn.CrossEntropyLoss()

In [24]:
model, optimizer, training_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(model, optimizer, training_dataloader, eval_dataloader, lr_scheduler)

In [25]:
num_train_epochs = 2
num_update_steps_per_epoch = len(training_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

In [26]:
print(train_dataset['input_ids'][0:3])

tensor([[21603,    10,  2575,  ...,     0,     0,     0],
        [21603,    10,    37,  ...,     0,     0,     0],
        [21603,    10,  4623,  ...,     0,     0,     0]])


In [27]:
for _ in range(10):
    batch = next(iter(training_dataloader))
    print(batch)


{'input_ids': tensor([[21603,    10,  2575,  ...,     0,     0,     0],
        [21603,    10,    37,  ...,     0,     0,     0],
        [21603,    10,  4623,  ...,     0,     0,     0],
        ...,
        [21603,    10,  1363,  ...,   701,    11,     1],
        [21603,    10, 26664,  ...,     8,  2076,     1],
        [21603,    10,  6946,  ...,     0,     0,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([[15921,    16, 26911,  ...,  -100,  -100,  -100],
        [   37,  4061,    13,  ...,  -100,  -100,  -100],
        [ 1331, 11548,  3201,  ...,  -100,  -100,  -100],
        ...,
        [12605,    31,     7,  ...,  -100,  -100,  -100],
        [  389,   850,    18,  ...,  -100,  -100,  -100],
        [   71,  1874,  2066,  ...,  -1

In [29]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in training_dataloader:
        inputs, targets = batch["input_ids"], batch["labels"] # Move tensors to the same device if not Accelerate

        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        accelerator.backward(loss)
        # loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/32 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 59.56 MiB is free. Process 83725 has 14.52 GiB memory in use. Of the allocated memory 14.36 GiB is allocated by PyTorch, and 32.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

# print(type(training_dataloader.dataset))
# print(len(training_dataloader.dataset))
# print(type(training_dataloader.dataset['input_ids'][0]))


# print(next(iter(training_dataloader)))


# for batch in training_dataloader:
#     print(batch)

In [ ]:

# for epoch in range(num_train_epochs):
#     model.train()
#     for step, batch in enumerate(training_dataloader.dataset['input_ids']):
       